<a href="https://colab.research.google.com/github/berkaykarlik/whoIsThatPokemon_CNN/blob/master/Atesv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount the Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Import Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
from tqdm import tqdm



Get the Images

In [11]:
path='/content/gdrive/My Drive/PokemonData'

classes = []
data = []
SIZE = 100

def get_data():
  class_no = 0
  for dir in tqdm(os.listdir(path)):
    classes.append(dir)
    dir_path = os.path.join(path,dir)
    for img in tqdm(os.listdir(dir_path)):
      try:
        img_array = cv2.imread(os.path.join(dir_path,img))
        resized_array = cv2.resize(img_array,(SIZE,SIZE))
        data.append([resized_array,class_no])
      except:
        pass
    class_no += 1
get_data()

Streaming output truncated to the last 5000 lines.
100%|██████████| 150/150 [1:16:59<00:00, 30.79s/it]


Seperate features& labels, convert feature set to numpy array



In [0]:
x = []
y = []
for features,label in data:
    x.append(features)
    y.append(label)

x = np.array(x).reshape(-1, SIZE, SIZE, 3)
y = np.array(y)

Split Data

In [13]:
test_images = []
test_labels = []
train_images = []
train_labels = []


testCnt = 0
for i in range(1,len(y)):
    if y[i] == y[i-1]:
        if testCnt < 5:
            test_images.append(x[i-1])
            test_labels.append(y[i-1])
        else:
            train_images.append(x[i-1])
            train_labels.append(y[i-1])
        testCnt = testCnt + 1
    else:
        testCnt = 0
    
test_images = np.array(test_images).reshape(-1, SIZE, SIZE, 3)
train_images = np.array(train_images).reshape(-1, SIZE, SIZE, 3)
test_labels = np.array(test_labels)
train_labels = np.array(train_labels)


print(train_images.shape,test_images.shape)

(5935, 100, 100, 3) (750, 100, 100, 3)


CNN

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [0]:
train_images, test_images = train_images / 255.0, test_images / 255.0

In [0]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(SIZE, SIZE ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(150)
])

In [25]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=100, 
                    validation_data=(test_images, test_labels))

Epoch 1/100
186/186 [==============================] - 2s 10ms/step - loss: 4.4343 - accuracy: 0.0785 - val_loss: 3.3976 - val_accuracy: 0.2107
Epoch 2/100
186/186 [==============================] - 2s 10ms/step - loss: 2.4319 - accuracy: 0.4081 - val_loss: 2.5348 - val_accuracy: 0.3933
Epoch 3/100
186/186 [==============================] - 2s 11ms/step - loss: 1.0830 - accuracy: 0.7242 - val_loss: 2.7713 - val_accuracy: 0.3960
Epoch 4/100
186/186 [==============================] - 2s 10ms/step - loss: 0.3809 - accuracy: 0.9068 - val_loss: 3.1386 - val_accuracy: 0.4000
Epoch 5/100
186/186 [==============================] - 2s 10ms/step - loss: 0.1135 - accuracy: 0.9749 - val_loss: 3.3862 - val_accuracy: 0.4187
Epoch 6/100
186/186 [==============================] - 2s 10ms/step - loss: 0.0556 - accuracy: 0.9897 - val_loss: 3.4193 - val_accuracy: 0.4307
Epoch 7/100
186/186 [==============================] - 2s 10ms/step - loss: 0.0461 - accuracy: 0.9943 - val_loss: 3.6006 - val_accuracy: